In [1]:
import pandas as pd
import json
import urllib
import datetime as dt
import time
import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
def query_station_status(url):
    


    with urllib.request.urlopen(url) as data_url:
        data = json.loads(data_url.read().decode())


    df = pd.DataFrame(data['data']['stations'])
    # drop inactive stations
    df = df[df.is_renting==1]
    df = df[df.is_returning==1]
    df = df.drop_duplicates(['station_id','last_reported'])
    df.last_reported = df.last_reported.map(lambda x: dt.datetime.utcfromtimestamp(x))
    df['time'] = data['last_updated']
    df.time = df.time.map(lambda x: dt.datetime.utcfromtimestamp(x))
    df = df.set_index('time')
    df.index = df.index.tz_localize('UTC')

    return df

In [13]:
url_station_bixi = "https://gbfs.velobixi.com/gbfs/fr/station_status.json"
df = query_station_status(url_station_bixi)
df = pd.pivot_table(df,columns='station_id',index='time',values='num_bikes_available')
df

station_id,1,10,100,101,102,103,104,105,106,107,...,9,90,91,92,94,95,96,97,98,99
time,,,,,,,,,,,,,,,,,,,,,
2022-05-12 13:04:54+00:00,22,5,16,5,17,11,3,12,12,1,...,6,0,0,21,16,22,2,3,14,2
